In [5]:
import os
file_names:list[str] = os.listdir('stations')
abs_path:list[str] = []
for name in file_names:
    if '每日各站進出站人數' in name:
        abs_path.append(os.path.abspath(f'stations/{name}')) #抓取檔案位置的絕對路徑

abs_path

['/workspaces/2024_02_20_2_4/lesson19/stations/每日各站進出站人數2022.csv',
 '/workspaces/2024_02_20_2_4/lesson19/stations/每日各站進出站人數2023.csv',
 '/workspaces/2024_02_20_2_4/lesson19/stations/每日各站進出站人數20190423-20191231.csv',
 '/workspaces/2024_02_20_2_4/lesson19/stations/每日各站進出站人數2021.csv',
 '/workspaces/2024_02_20_2_4/lesson19/stations/每日各站進出站人數2020.csv']

In [8]:
import pandas as pd
from pandas import DataFrame

all_df:list[DataFrame] = [pd.read_csv(path) for path in abs_path]
display(all_df)

[       trnOpDate  staCode  gateInComingCnt  gateOutGoingCnt
 0       20220101      900             8645             8526
 1       20220101      910              996             1243
 2       20220101      920             1369             1711
 3       20220101      930             3663             4535
 4       20220101      940             1532             1845
 ...          ...      ...              ...              ...
 86547   20221231     7360             3984             3893
 86548   20221231     7361               92              115
 86549   20221231     7362              131              159
 86550   20221231     7380              494              426
 86551   20221231     7390              313              274
 
 [86552 rows x 4 columns],
        trnOpDate  staCode  gateInComingCnt  gateOutGoingCnt
 0       20230101      900             8030             8241
 1       20230101      910             1017             1326
 2       20230101      920             1400             

In [12]:
inOut_df = pd.concat(all_df) #利用concat將5個檔案整合連結成一個
inOut_df.columns = ['乘車日', '車站代碼', '進站人數', '出站人數']
inOut_df

,乘車日,車站代碼,進站人數,出站人數
0,20220101,900,8645,8526
1,20220101,910,996,1243
2,20220101,920,1369,1711
3,20220101,930,3663,4535
4,20220101,940,1532,1845
...,...,...,...,...
86927,20201231,7360,3523,3602
86928,20201231,7361,3,18
86929,20201231,7362,28,34
86930,20201231,7380,609,604


In [13]:
import requests
import pandas as pd
#車站基本資料集 - https://data.gov.tw/dataset/33425
url2 = 'https://ods.railway.gov.tw/tra-ods-web/ods/download/dataResource/0518b833e8964d53bfea3f7691aea0ee'
response = requests.get(url2)
if response.status_code == 200:
    station_data = response.json()

station_df = pd.DataFrame(station_data,columns=['stationCode','stationName'])
station_df.columns = ['車站代碼','中文站名']
station_df['車站代碼'] = station_df['車站代碼'].astype(int)
station_df

,車站代碼,中文站名
0,900,基隆
1,910,三坑
2,920,八堵
3,930,七堵
4,940,百福
...,...,...
237,7350,猴硐
238,7360,瑞芳
239,7361,海科館
240,7380,四腳亭


In [14]:
station_all_df = pd.merge(inOut_df, station_df, on='車站代碼', how='left')
station_all_df

,乘車日,車站代碼,進站人數,出站人數,中文站名
0,20220101,900,8645,8526,基隆
1,20220101,910,996,1243,三坑
2,20220101,920,1369,1711,八堵
3,20220101,930,3663,4535,七堵
4,20220101,940,1532,1845,百福
...,...,...,...,...,...
406756,20201231,7360,3523,3602,瑞芳
406757,20201231,7361,3,18,海科館
406758,20201231,7362,28,34,NaN
406759,20201231,7380,609,604,四腳亭


In [15]:
station_all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406761 entries, 0 to 406760
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   乘車日     406761 non-null  int64 
 1   車站代碼    406761 non-null  int64 
 2   進站人數    406761 non-null  int64 
 3   出站人數    406761 non-null  int64 
 4   中文站名    405059 non-null  object
dtypes: int64(4), object(1)
memory usage: 15.5+ MB


In [16]:
#移除內容值為 Nan的欄位
station_all_df1 = station_all_df.dropna()
station_all_df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 405059 entries, 0 to 406760
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   乘車日     405059 non-null  int64 
 1   車站代碼    405059 non-null  int64 
 2   進站人數    405059 non-null  int64 
 3   出站人數    405059 non-null  int64 
 4   中文站名    405059 non-null  object
dtypes: int64(4), object(1)
memory usage: 18.5+ MB


In [21]:
#將乘車日改為Datatime
station_all_df1['乘車日'] = pd.to_datetime(station_all_df1['乘車日'].astype(str)) #轉datatime時，astype可有可無
station_all_df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 405059 entries, 0 to 406760
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   乘車日     405059 non-null  datetime64[ns]
 1   車站代碼    405059 non-null  int64         
 2   進站人數    405059 non-null  int64         
 3   出站人數    405059 non-null  int64         
 4   中文站名    405059 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 18.5+ MB


/tmp/ipykernel_43649/3389804076.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_all_df1['乘車日'] = pd.to_datetime(station_all_df1['乘車日'])


In [23]:
station_all_df2 = station_all_df1.sort_values(by='乘車日') #sort_values做昇序排列
station_all_df2

,乘車日,車站代碼,進站人數,出站人數,中文站名
173161,2019-04-23,900,8442,7743,基隆
173313,2019-04-23,5070,533,424,南州
173314,2019-04-23,5080,15,14,鎮安
173315,2019-04-23,5090,368,325,林邊
173316,2019-04-23,5100,95,95,佳冬
...,...,...,...,...,...
173029,2023-12-31,4080,12822,11455,嘉義
173030,2023-12-31,4090,801,625,水上
173031,2023-12-31,4100,229,190,南靖
173019,2023-12-31,3435,125,356,水里


In [26]:
station_array = station_all_df2.values
station_array[:,0] #抓取第0列位的資料 

array([Timestamp('2019-04-23 00:00:00'), Timestamp('2019-04-23 00:00:00'),
       Timestamp('2019-04-23 00:00:00'), ...,
       Timestamp('2023-12-31 00:00:00'), Timestamp('2023-12-31 00:00:00'),
       Timestamp('2023-12-31 00:00:00')], dtype=object)

In [28]:
station_array[:,[0,2,3]] #抓取第0、2、3列的資料

array([[Timestamp('2019-04-23 00:00:00'), 8442, 7743],
       [Timestamp('2019-04-23 00:00:00'), 533, 424],
       [Timestamp('2019-04-23 00:00:00'), 15, 14],
       ...,
       [Timestamp('2023-12-31 00:00:00'), 229, 190],
       [Timestamp('2023-12-31 00:00:00'), 125, 356],
       [Timestamp('2023-12-31 00:00:00'), 2040, 1863]], dtype=object)

In [29]:
station_all_df2.query('乘車日=="2023"') #比對符合2023的資料列出

/tmp/ipykernel_43649/2084214848.py:1: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  station_all_df2.query('乘車日=="2023"')


,乘車日,車站代碼,進站人數,出站人數,中文站名
86726,2023-01-01,6070,765,598,池上
86725,2023-01-01,6060,2,3,海端
86727,2023-01-01,6080,155,140,富里
86728,2023-01-01,6090,4,6,東竹
86729,2023-01-01,6100,19,9,東里
...,...,...,...,...,...
86569,2023-01-01,1070,7002,8035,鶯歌
86570,2023-01-01,1080,28683,33844,桃園
86571,2023-01-01,1090,6724,8919,內壢
86555,2023-01-01,930,3558,4398,七堵
